In [ ]:
# Dataset from https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020

In [ ]:
import pandas as pd

In [ ]:
# import the csv to a dataframe
df = pd.read_csv('../data/raw/marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
bin_dataset = df[['Uniq Id','Shipping Weight','Selling Price']]

# rename columns
bin_dataset = bin_dataset.rename(columns={'Uniq Id':'id','Shipping Weight':'weight','Selling Price':'price'})

In [ ]:
bin_dataset.info()

In [ ]:
# Drop null values from the table.
bin_dataset = bin_dataset.dropna(subset=['weight', 'price'])
bin_dataset.info()

In [ ]:
weight_counts = bin_dataset.weight.str.extract(r'(pounds|ounces)').value_counts()
pounds_rows = weight_counts['pounds']
ounces_rows = weight_counts['ounces']
total_rows = weight_counts.sum()
print(f"The match found {total_rows} rows, of this:\n{pounds_rows} rows contain the word 'pounds'\n{ounces_rows} rows contain the word 'ounces'")

In [ ]:
# to get these to a uniform weight we need to extract the numeric values from the strings.
# Convert ounces to kilograms (1 pound = 0.0283495 kilograms).
# Convert pounds to kilograms (1 pound = 0.453592 kilograms).

def convert_to_kilograms(weight_str):
    # Remove any non-numeric characters, including commas
    numeric_str = ''.join(c for c in weight_str if c.isdigit() or c == '.')

    try:
        value = float(numeric_str)
    except ValueError:
        return None  # Return None if the numeric part cannot be converted

    parts = weight_str.split()
    unit = parts[-1].lower()  # Get the last part as the unit in lowercase

    if unit == 'pounds':
        return value * 0.453592
    elif unit == 'ounces':
        return value * 0.0283495
    else:
        return None  # Return None for unknown units


In [ ]:
bin_dataset['weight_in_kg'] = bin_dataset['weight'].apply(convert_to_kilograms)

In [ ]:
bin_dataset

In [ ]:
# We apply the convert_to_kilograms function to the 'weight' column to populate the 'weight_in_kg' column.
# We use the .isna() method to create a boolean mask that identifies rows where 'weight_in_kg' is None (indicating errors).
# We use this boolean mask to select and display only the rows with errors.

# Apply the conversion function to the 'Weight' column
bin_dataset['weight_in_kg'] = bin_dataset['weight'].apply(convert_to_kilograms)

# Filter rows with errors (where 'Weight_in_kg' is None)
rows_with_errors = bin_dataset[bin_dataset['weight_in_kg'].isna()]

# Display rows with errors
print(rows_with_errors)

In [ ]:
# Drop rows with invalid or unknown units
bin_dataset = bin_dataset.dropna(subset=['weight_in_kg'])

In [ ]:
bin_dataset['price']

In [ ]:
def convert_to_number(price_str):
    # Remove non-numeric characters and currency symbols
    numeric_str = ''.join(c for c in price_str if c.isdigit() or c == '.')

    try:
        return float(numeric_str)
    except ValueError:
        return None 

In [ ]:
bin_dataset['cleaned_price'] = bin_dataset['price'].apply(convert_to_number)
# Filter rows with errors (where 'price' is None)
rows_with_errors = bin_dataset[bin_dataset['cleaned_price'].isna()]

# Display rows with errors
rows_with_errors

In [ ]:
# We could clean these up a bit, but it's a small percentage of the dataset. 
# So we'll just exclude these rows
bin_dataset = bin_dataset.dropna(subset=['cleaned_price'])

In [ ]:
# Now that the dataset is a bit cleaner, we're going to drop the old columns.
# Axis 1 is specified to indicate that it's columns we want to remove
bin_dataset = bin_dataset.drop(['price', 'weight'], axis=1)

In [ ]:
bin_dataset.head()

In [ ]:
# Just as we did earlier, we're going to rename the columns
bin_dataset = bin_dataset.rename(columns={'weight_in_kg':'weight_kg','cleaned_price':'price'})

In [ ]:
bin_dataset.head()

In [ ]:
# Now that this has been cleaned up a bit, we can export it for future use.
bin_dataset.to_csv('../data/interim/marketing_sample_for_amazon_com-ecommerce__20200101_20200131__weight_price.csv',index=False)

In [ ]:
bin_dataset.info()